In [29]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
data0 = pd.read_csv("insurance_data_preprocessed.csv")


In [31]:
import pandas as pd
import numpy as np

def evaluate_insurance_profitability(df):
    """
    评估保险盈利状况
    """
    results = {}
    
    # 基础指标计算
    total_premium = (df['annual_premium'] * df['policy_term_years']).sum()
    total_claims_paid = df['total_claims_paid'].sum() 
    total_medical_cost = df['annual_medical_cost'].sum()
    
    # 1. 赔付率 (Loss Ratio) - 最重要的指标
    loss_ratio = total_claims_paid / total_premium
    results['loss_ratio'] = loss_ratio
    
    # 2. 医疗成本覆盖率
    medical_cost_coverage = total_claims_paid / total_medical_cost
    results['medical_cost_coverage'] = medical_cost_coverage
    
    # 3. 承保利润
    underwriting_profit = total_premium - total_claims_paid
    results['underwriting_profit'] = underwriting_profit
    
    # 4. 承保利润率
    underwriting_margin = underwriting_profit / total_premium
    results['underwriting_margin'] = underwriting_margin
    
    return results

# 应用评估
profit_metrics = evaluate_insurance_profitability(data0)
profit_metrics

{'loss_ratio': 0.43090498665899823,
 'medical_cost_coverage': 0.45836468290484983,
 'underwriting_profit': 175697179.41999996,
 'underwriting_margin': 0.5690950133410018}

## 用户角度

In [32]:
def correct_customer_analysis(df):
    """
    正确的用户最优投保策略分析
    """
    term_analysis = df.groupby('policy_term_years').agg({
        'annual_premium': 'mean',
        'total_claims_paid': 'mean',  # 这是年赔偿
        'annual_medical_cost': 'mean',
        'policy_changes_last_2yrs': 'mean'
    }).reset_index()
    
    # 正确计算：年赔偿 / 年保费
    term_analysis['annual_value_score'] = term_analysis['total_claims_paid'] / term_analysis['annual_premium']
    
    # 其他重要指标
    term_analysis['coverage_ratio'] = term_analysis['total_claims_paid'] / term_analysis['annual_medical_cost']
    term_analysis['out_of_pocket_ratio'] = 1 - term_analysis['coverage_ratio']
    
    return term_analysis

# 正确分析
correct_analysis = correct_customer_analysis(data0)
print("=== 正确的分析结果 ===")
print(correct_analysis[['policy_term_years', 'annual_premium', 'total_claims_paid', 'annual_value_score', 'coverage_ratio']])

=== 正确的分析结果 ===
   policy_term_years  annual_premium  total_claims_paid  annual_value_score  \
0                  1      584.184240        1392.275974            2.383282   
1                  2      588.136198        1402.643820            2.384896   
2                  3      579.543937        1372.888914            2.368913   
3                  4      595.874702        1428.260560            2.396914   
4                  5      580.272633        1360.212058            2.344091   
5                  6      588.307298        1389.162301            2.361287   
6                  7      587.568167        1420.704424            2.417940   
7                  8      585.308956        1387.098031            2.369856   
8                  9      586.238528        1394.964557            2.379517   
9                 10      588.114068        1397.089714            2.375542   

   coverage_ratio  
0        0.462159  
1        0.458347  
2        0.457727  
3        0.460695  
4        0.454

In [33]:
correct_analysis

,policy_term_years,annual_premium,total_claims_paid,annual_medical_cost,policy_changes_last_2yrs,annual_value_score,coverage_ratio,out_of_pocket_ratio
0,1,584.184240,1392.275974,3012.545362,0.051085,2.383282,0.462159,0.537841
1,2,588.136198,1402.643820,3060.223602,0.049713,2.384896,0.458347,0.541653
2,3,579.543937,1372.888914,2999.362993,0.048811,2.368913,0.457727,0.542273
3,4,595.874702,1428.260560,3100.230672,0.052281,2.396914,0.460695,0.539305
4,5,580.272633,1360.212058,2994.582761,0.055468,2.344091,0.454224,0.545776
5,6,588.307298,1389.162301,3061.913224,0.048750,2.361287,0.453691,0.546309
6,7,587.568167,1420.704424,3053.366625,0.049921,2.417940,0.465291,0.534709
7,8,585.308956,1387.098031,3031.054644,0.049415,2.369856,0.457629,0.542371
8,9,586.238528,1394.964557,3044.433096,0.054739,2.379517,0.458202,0.541798
9,10,588.114068,1397.089714,3065.462806,0.048969,2.375542,0.455752,0.544248


In [34]:
a = data0.annual_medical_cost - data0.deductible - data0.copay - data0.total_claims_paid
b = data0.annual_medical_cost  - data0.total_claims_paid
a/b

0         0.549762
1        -1.277185
2         0.932124
3         0.269854
4         0.648006
           ...    
95389     0.606190
95390    -2.457943
95391     3.530074
95392   -10.542857
95393    -1.190251
Length: 95394, dtype: float64

In [36]:
(b <0).sum()

4100

In [41]:
data = data0[data0.annual_medical_cost - data0.deductible - data0.copay - data0.total_claims_paid >0]
a = data.annual_medical_cost - data.deductible - data.copay - data.total_claims_paid
b = data.annual_medical_cost  - data.total_claims_paid

In [42]:
a/b

0        0.549762
2        0.932124
3        0.269854
4        0.648006
5        0.708017
           ...   
95384    0.888136
95385    0.328991
95386    0.877033
95387    0.519194
95389    0.606190
Length: 48850, dtype: float64